In [1]:
# notebook setup
import datetime as dt
import os
import sys
from pathlib import Path
p = Path().resolve().parents[1]
sys.path.append(str(p))
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd

import gscap.weather as weather

InteractiveShell.ast_node_interactivity = 'all'

### Get a daily weather report by latitude and longitude

In [2]:
now = dt.datetime.now()
today = dt.datetime(year=now.year, month=now.month, day=now.day); today

report = weather.weather_report((32.503082, -97.187339, today))
pd.DataFrame(report.get('report')).head()

datetime.datetime(2022, 2, 20, 0, 0)

C:\Users\calvi\OneDrive\Documents\thesi\gSCAP\.gscapConfig


,date,lat,lon,zipcode,base,visibility,dt,timezone,id,name,...,main_humidity,wind_speed,wind_deg,wind_gust,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,2022-02-20,32.503082,-97.187339,76061,stations,10000,1645423174,-21600,4709013,Mansfield,...,78,5.66,170,7.72,0,2,2012737,US,1645362398,1645402718


If you notice above, report comes back from the weather module as a dictionary. It contains three attributes: hits, misses, and the report itself. `hits` tells you how many times you were able to retrieve a request from the cache while `misses` tells you how many API calls were made. This is extremely valuable for tracking expenses. You can view them as shown in the next cell.

In [3]:
report['hits']
report['misses']

0

1

### Or get a report by zipcode

In [4]:
report = weather.weather_report((today, 76092))
pd.DataFrame(report.get('report')).head()

C:\Users\calvi\OneDrive\Documents\thesi\gSCAP\.gscapConfig


,date,lat,lon,zipcode,base,visibility,dt,timezone,id,name,...,main_humidity,wind_speed,wind_deg,wind_gust,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,2022-02-20,32.95987,-97.14843,76092,stations,10000,1645423180,-21600,4733313,Southlake,...,75,4.02,161,7.6,100,2,2003388,US,1645362417,1645402680


### If you have a series of requests to make, just include a list of a requests. You can set `n_jobs` to use as many cores as you want. 
For example, the following request makes four simultaneous calls to Dark Sky retrieving daily weather summaries by zipcode or lat-lon pairs, and DateTimes in any order. The only order that must be maintained is that the latitude must come before the longitude.

In [6]:
requests = [
    (34.41, -86.92, today-dt.timedelta(days=4)),
    (today-dt.timedelta(days=3), 32.11, -43.92),
    (2.11, today-dt.timedelta(days=4), -52.32),
]

report = weather.weather_report(requests)

pd.DataFrame(report.get('report')).head()
print(f'hits: {report["hits"]}, misses: {report["misses"]}')

,date,lat,lon,zipcode,base,visibility,dt,timezone,id,name,...,main_grnd_level,wind_speed,wind_deg,wind_gust,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,2022-02-16,34.41,-86.92,35640,stations,10000,1645423220,-21600,4066449,Hartselle,...,1002,3.21,151,9.68,96,2.0,19179.0,US,1645446388,1645486586
1,2022-02-17,32.11,-43.92,4631,stations,10000,1645423225,-10800,0,,...,1028,7.14,132,8.08,59,NaN,NaN,NaN,1645435935,1645476401
2,2022-02-16,2.11,-52.32,33070,stations,10000,1645423234,-10800,3380394,Saint-Joseph,...,989,0.75,36,3.13,98,NaN,NaN,GF,1645436475,1645479893


hits: 0, misses: 3


### The default summarization is at the daily level as shown above. To return the results at they hourly level set `summarize = 'none'`

In [ ]:
report = weather.weather_report(requests, summarize='none')
report = pd.DataFrame(report.get('report'))
report.head(50)